In [155]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from ast import literal_eval


In [156]:
var_dict = {
    'T':'Time_Spend_Company',
    'D':'Department',
    'NP':'Number_Projects',
    'P':'Promotion_Last_5_Years',
    'L':'Left',
    'H':'Average_Montly_Hours',
    'WA':'Work_Accident',
    'LE':'Last_Evaluation',
    'S':'Salary',
    'SL':'Satisfaction_Level'
}

def item2dict(item):
    ## splitto la stringa
    item_split = item.split('_')
    # definisco la lista variabile,valore
    item_list = {var_dict[item_split[1]] : item_split[0]}
    return item_list


In [157]:
def df2df_set(df):
    """funzione che converte 
    le stringhe relative alle regole in 
    liste di dizionari """

    df_new = df.copy()

    df_new = df_new.drop(['Antecedent','Consequent'],axis=1)

    Antecedent=[]
    for i,item in enumerate(df.Antecedent):
        Antecedent.append(set(literal_eval(item)))

    df_new=df_new.assign(Antecedent=Antecedent)

    Consequent=[]
    for i,item in enumerate(df.Consequent):
        Consequent.append(item2dict(item))

    df_new=df_new.assign(Consequent=Consequent)
        
    return df_new        


# Analisi regole Per LEFT

In [229]:
df = pd.read_csv("../../data/rules/rules_supp5_conf50.csv")
df.shape
#df_filtered = df.query('Supp>0.2') # perchè non è maggiore di 20???
#df_filtered = df_filtered.query('Lift>1')
df_left = df.query(" Consequent=='Y_L' and Lift>1")

df_left.to_csv("../../code/rules/df_left_all.csv")
df_left.describe()

# converto in dataset di set
df_left_set=df2df_set(df_left)
df_left_set.head()

# aggiungo la lunghezza della parte antecedente
N_items=[]
ant = df_left_set.Antecedent
N_items = []
for i,ant in enumerate(df_left_set.Antecedent):
    N_items.append(len(df_left_set.Antecedent.iloc[i]))


df_left_set['N_items']=N_items
df_left_set.head()

    Supp  Conf  Lift                     Antecedent       Consequent  N_items
79  0.05  0.77  3.25  {intensive_H, N_P, N_WA, 5_T}  {u'Left': u'Y'}        4
82  0.05  0.77  3.24       {intensive_H, N_WA, 5_T}  {u'Left': u'Y'}        3
85  0.05  0.74  3.11        {intensive_H, N_P, 5_T}  {u'Left': u'Y'}        3
87  0.05  0.74  3.09             {intensive_H, 5_T}  {u'Left': u'Y'}        2
90  0.05  0.62  2.58               {N_P, N_WA, 5_T}  {u'Left': u'Y'}        3

# Inizio pruning delle regole

In [230]:
# pre-filtering
df_left_filtered=df_left_set.query('Conf>0.5')
df_left_filtered.describe()

             Supp        Conf        Lift     N_items
count  264.000000  264.000000  264.000000  264.000000
mean     0.069924    0.811477    3.410152    4.121212
std      0.021343    0.131180    0.551107    1.293625
min      0.050000    0.510000    2.130000    1.000000
25%      0.050000    0.720000    3.030000    3.000000
50%      0.060000    0.850000    3.580000    4.000000
75%      0.090000    0.920000    3.880000    5.000000
max      0.150000    0.980000    4.100000    8.000000

In [253]:
# rimuovo quelli le regole contenenti quelli che non hanno avuto incidenti
data= df_left_filtered.copy()

indici_remove=[]
for i in range(data.shape[0]):
    if set(['N_WA']).issubset( data.iloc[i].Antecedent ):
        indici_remove.append(i)
#        print i
        #         print data.iloc[i].Antecedent
print len(indici_remove)
data.drop(data.index[indici_remove],inplace=True)
data.describe()
data_clean=data
data_clean.head()
del(data)

136


In [275]:
data=data_clean.query('Conf>0.9')
data=data.sort_values(by='N_items')
data_pruned=data.copy()

#print data_pruned.head()
# ordino

# prendo la prima
row=0
while row<data_pruned.shape[0] :
    rule = data.iloc[row].Antecedent
    #print row
    indici_remove=[]
    for i in range((row+1),(data_pruned.shape[0])):
        successiva = data_pruned.iloc[i].Antecedent
        if rule.issubset(successiva):
            indici_remove.append(i)

    data_pruned.drop(data_pruned.index[[indici_remove]],inplace=True)
    del(indici_remove)
    
    row+=1



In [276]:
data_pruned.describe()

            Supp       Conf       Lift    N_items
count  11.000000  11.000000  11.000000  11.000000
mean    0.070000   0.924545   3.882727   4.181818
std     0.020494   0.010357   0.043149   0.750757
min     0.050000   0.910000   3.800000   3.000000
25%     0.055000   0.920000   3.870000   4.000000
50%     0.060000   0.920000   3.880000   4.000000
75%     0.090000   0.930000   3.890000   5.000000
max     0.100000   0.940000   3.960000   5.000000

In [263]:
df_left[data_]
data_pruned.index
#data_pruned.to_csv("../../data/best_rules.csv")

Int64Index([1351, 1095, 2817, 1255, 1235, 1371, 1963,  185, 2673, 2797, 2793,
            2629,  181,  145, 2697, 2669, 2624, 2692],
           dtype='int64')

In [274]:
data_best_rules=df_left.loc[data_pruned.index]
data_pruned.to_csv("../../data/rules/best_rules.csv")

